# Phase III Code

Phase III will utilize code to determine the dimensions of the anicut structure, as well as the percolation pond. The Runoff data from Phase II will be used in order to determine the Max runoff within the past several years. Using the Max runoff value, the volume of the percolation pond will be determined. The height will be determined from analysis of the runoff, and the length then determined from that. The final area of the pond will then be determined. 

In [17]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import os

In [18]:
Runoff = pd.read_csv('Precipitation.csv')
Runoff.head().T

#Reading the runoff data produced in Phase II

,0,1,2,3,4
SUBDIVISION,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan
YEAR,1901,1902,1903,1904,1905
JAN,6.7,0,1.7,3.8,6.3
FEB,0,0,1.3,2.9,4.8
MAR,1.1,0,5.5,16.3,0.7
APR,0,0.5,0,0.7,1.3
MAY,6.1,4,4.2,11.4,0.3
JUN,3,49.1,2.7,14.6,4.9
JUL,79,27,154.8,39.8,30.1
AUG,59.2,71.3,87.1,45.6,0.6


In [19]:
#Adding the summation of expected runoff for each upcoming year and adding it as its own column
annual = [ row.JAN + row.FEB + row.MAR + row.APR + row.MAY + row.JUN + row.JUL + row.AUG + row.SEP + row.OCT + row.NOV + row.DEC for index, row in Runoff.iterrows() ]
Runoff['ANNUAL'] = annual
Runoff.head().T

,0,1,2,3,4
SUBDIVISION,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan,West Rajasthan
YEAR,1901,1902,1903,1904,1905
JAN,6.7,0,1.7,3.8,6.3
FEB,0,0,1.3,2.9,4.8
MAR,1.1,0,5.5,16.3,0.7
APR,0,0.5,0,0.7,1.3
MAY,6.1,4,4.2,11.4,0.3
JUN,3,49.1,2.7,14.6,4.9
JUL,79,27,154.8,39.8,30.1
AUG,59.2,71.3,87.1,45.6,0.6


In [20]:
#Choosing the maximum capacity yield projected in the next 50 years
Runoff['ANNUAL'].max()

768.8

In [21]:
AE = pd.read_csv('Anicut Elevation.csv')
AE

#AE = Anicut Elevation

,Elevation,Distance L,Distance R,Distance (English),Distance (Metric),Anicut Height (English),Anicut Height (Metric),Anicut Height (Rounded)
0,Ft,Ft,Ft,Ft,m,Ft,m,m
1,1228,606,742,136,41,1,0.30,1
2,1229,550,782,232,71,2,0.61,1
3,1230,495,823,328,100,3,0.91,1
4,1231,443,864,421,128,4,1.22,2
5,1232,393,903,510,155,5,1.52,2
6,1233,372,941,569,173,6,1.83,2
7,1234,353,969,616,188,7,2.13,3
8,1235,330,991,661,201,8,2.44,3
9,1236,310,1005,695,212,9,2.74,3


In [22]:
AE1 = AE.drop(['Distance L', 'Distance R', 'Distance (English)','Anicut Height (English)','Anicut Height (Rounded)'], axis=1)
AE1

,Elevation,Distance (Metric),Anicut Height (Metric)
0,Ft,m,m
1,1228,41,0.30
2,1229,71,0.61
3,1230,100,0.91
4,1231,128,1.22
5,1232,155,1.52
6,1233,173,1.83
7,1234,188,2.13
8,1235,201,2.44
9,1236,212,2.74


In [30]:
Height = 1

In [31]:
if Height == 0.5:
    Length = 71
    print('Length of Dam = 71 m')
elif Height == 1:
    Length = 128
    print('Length of Dam = 128 m')
elif Height == 1.5:
    Length = 155
    print('Length of Dam = 155 m')
elif Height == 2:
    Length = 188
    print('Length of Dam = 188 m')
elif Height == 2.5: 
    Length = 212
    print('Length of Dam = 212 m')

Length of Dam = 128 m


In [32]:
Surface_Area = Height*Length
Surface_Area

128

In [33]:
Reservoir_Length=Runoff['ANNUAL'].max()/Surface_Area
Reservoir_Length

6.00625